In [32]:
import pandas as pd
combined_csv = '/app/combined.csv'
combined_run_df = pd.read_csv(combined_csv)

In [33]:
company_counts = combined_run_df.groupby('Firma')['Nr'].nunique().reset_index()
# Sort the companies based on the number of runners in descending order
sorted_companies = company_counts.sort_values('Nr', ascending=False)
sorted_companies = sorted_companies.rename(columns={'Nr': 'Läufer'})
# Reset the index of the sorted dataframe
sorted_companies.reset_index(drop=True, inplace=True)

In [34]:
summary_stats = combined_run_df.groupby('Firma')['Zeit'].agg(['min', 'max', 'mean', 'std']).rename(columns={'min': 'Min_Zeit', 'max': 'Max_Zeit', 'mean': 'Ds_Zeit'})
gender_counts = combined_run_df.groupby('Firma')['Geschlecht'].value_counts().unstack(fill_value=0).rename(columns={'W': 'Anzahl_Frauen', 'M': 'Anzahl_Männer'})
sorted_companies = pd.merge(sorted_companies, summary_stats, on='Firma')
sorted_companies = pd.merge(sorted_companies, gender_counts, on='Firma')
sorted_companies['Ds_Tempo'] = 6.3 * 3600 / sorted_companies['Ds_Zeit']
sorted_companies['Anteil_Männer'] = sorted_companies['Anzahl_Männer'].divide(sorted_companies['Läufer'])
sorted_companies['Anteil_Frauen'] = sorted_companies['Anzahl_Frauen'].divide(sorted_companies['Läufer'])

In [35]:
sorted_companies.head()

,Firma,Läufer,Min_Zeit,Max_Zeit,Ds_Zeit,std,Anzahl_Männer,Anzahl_Frauen,Ds_Tempo,Anteil_Männer,Anteil_Frauen
0,ArcelorMittal Bremen GmbH,149,1435.7,3824.5,2050.810067,347.973536,125,24,11.059045,0.838926,0.161074
1,Airbus Operations GmbH,123,1339.0,3493.4,2068.026829,416.965796,101,22,10.966976,0.821138,0.178862
2,ATLAS ELEKTRONIK GmbH,108,1397.4,3643.3,2170.349074,426.764891,86,22,10.449932,0.796296,0.203704
3,Rheinmetall Electronics GmbH,98,1339.5,3275.5,2161.122449,337.206604,84,14,10.494546,0.857143,0.142857
4,Unternehmensgruppe Lürssen,79,1509.9,2747.1,2070.739241,331.365855,69,10,10.952610,0.873418,0.126582


In [36]:
sorted_companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357 entries, 0 to 356
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Firma          357 non-null    object 
 1   Läufer         357 non-null    int64  
 2   Min_Zeit       357 non-null    float64
 3   Max_Zeit       357 non-null    float64
 4   Ds_Zeit        357 non-null    float64
 5   std            338 non-null    float64
 6   Anzahl_Männer  357 non-null    int64  
 7   Anzahl_Frauen  357 non-null    int64  
 8   Ds_Tempo       357 non-null    float64
 9   Anteil_Männer  357 non-null    float64
 10  Anteil_Frauen  357 non-null    float64
dtypes: float64(7), int64(3), object(1)
memory usage: 30.8+ KB


In [37]:
sorted_companies[sorted_companies.drop('std', axis=1).isna().any(axis=1)]

,Firma,Läufer,Min_Zeit,Max_Zeit,Ds_Zeit,std,Anzahl_Männer,Anzahl_Frauen,Ds_Tempo,Anteil_Männer,Anteil_Frauen


In [38]:
sorted_companies.to_parquet('/app/companies.parquet', index=False)